In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用对象检测的Vertex AI Model Garden MediaPipe

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_object_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_object_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_mediapipe_object_detection.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI工作台中打开
    </a>
  </td>
</table>

**_注意_**: 该笔记本已在以下环境中进行测试：

* Python版本 = 3.9

**注意**：此Colab中链接的检查点和数据集不是由谷歌拥有或分发的，而是由第三方提供。在使用检查点和数据之前，请先阅读第三方提供的条款和条件。

## 概述

本笔记本演示了如何在Vertex AI模型花园中使用[MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker)。

### 目标

* 训练新模型
  * 将输入数据转换为训练格式
  * 创建[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)来训练新模型
  * 导出模型

* 清理资源

### 成本

此教程使用Google Cloud的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

在你开始之前

只有Colab才能运行
请运行以下命令来安装依赖项并在Colab上进行Google Cloud身份验证。

In [ ]:
! pip3 install --upgrade pip

import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

#### 设置您的项目ID

**如果您不知道您的项目ID**，请参阅支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import json
import os
from datetime import datetime

import tensorflow
from google.cloud import aiplatform

### 初始化用于 Python 的 Vertex AI SDK

为您的项目初始化用于 Python 的 Vertex AI SDK。

In [ ]:
now = datetime.now().strftime("%Y%m%d-%H%M%S")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temp/%s" % now)

EVALUATION_RESULT_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "evaluation")
EVALUATION_RESULT_OUTPUT_FILE = os.path.join(
    EVALUATION_RESULT_OUTPUT_DIRECTORY, "evaluation.json"
)

EXPORTED_MODEL_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "model")
EXPORTED_MODEL_OUTPUT_FILE = os.path.join(
    EXPORTED_MODEL_OUTPUT_DIRECTORY, "model.tflite"
)

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### 定义培训机器规格

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "mediapipe_object_detector_%s" % now
TRAINING_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/mediapipe-train"
TRAINING_MACHINE_TYPE = "n1-highmem-16"
TRAINING_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAINING_ACCELERATOR_COUNT = 2

训练您自定义的模型##

### 为训练准备输入数据

对目标检测模型进行微调需要一个包含你希望完成模型能够识别的项目或类别的数据集。你可以通过从公共数据集中筛选出与你的用例相关的类别、编制自己的数据集，或者两者结合的方式来实现。数据集可以显著小于从头开始训练新模型所需的数据量。例如，用于训练许多参考模型的 [COCO](https://cocodataset.org/) 数据集包含数十万张图像，涵盖了 91 类对象。使用 Model Maker 进行迁移学习可以通过较小的数据集对现有模型进行微调，根据你的推断精度目标，模型仍然可以表现良好。本指南使用包含 2 种安卓小人玩偶（或者 2 类）的较小数据集，共有 62 张训练图像。

你可以重复使用现有数据集，如 `gs://mediapipe-tasks/object_detector/android_figurine` 来微调模型。该目录包含两个子目录，分别为训练集和验证集，分别位于 android_figurine/train 和 android_figurine/validation 目录中。每个训练集和验证集都遵循下面描述的 COCO 数据集的格式。如果你使用自己的数据集，请确保符合格式规范后再上传到 Google Cloud Storage。

### 支持的数据集格式
Model Maker 目标检测 API 支持读取以下数据集格式：

#### COCO 格式
COCO 数据集格式有一个 `data` 目录用于存储所有图像，还有一个包含所有图像对象注释的 `labels.json` 文件。
```
<dataset_dir>/
  data/
    <img0>.<jpg/jpeg>
    <img1>.<jpg/jpeg>
    ...
  labels.json
```
其中 `labels.json` 的格式如下：
```
{
  "categories":[
    {"id":1, "name":<cat1_name>},
    ...
  ],
  "images":[
    {"id":0, "file_name":"<img0>.<jpg/jpeg>"},
    ...
  ],
  "annotations":[
    {"id":0, "image_id":0, "category_id":1, "bbox":[x-左上, y-左上, 宽度, 高度]},
    ...
  ]
}
```

#### PASCAL VOC 格式
PASCAL VOC 数据集格式也有一个 `data` 目录用于存储所有图像，不过每个图像的注释被拆分为相应的 xml 文件存在 `Annotations` 目录中。
```
<dataset_dir>/
  data/
    <file0>.<jpg/jpeg>
    ...
  Annotations/
    <file0>.xml
    ...
```
其中 xml 文件格式如下：
```
<annotation>
  <filename>file0.jpg</filename>
  <object>
    <name>kangaroo</name>
    <bndbox>
      <xmin>233</xmin>
      <ymin>89</ymin>
      <xmax>386</xmax>
      <ymax>262</ymax>
    </bndbox>
  </object>
  <object>
    ...
  </object>
  ...
</annotation>
```

### 配置训练数据集

完成数据准备后，您可以开始微调模型以识别由训练数据定义的新对象或类。以下说明将使用前一部分准备的数据来微调一个目标检测模型，以识别两种类型的安卓人偶。

如果您没有单独的测试数据集，可以将测试数据的路径留空。

In [ ]:
training_data_path = "gs://mediapipe-tasks/object_detector/android_figurine/train"  # @param {type:"string"}
validation_data_path = "gs://mediapipe-tasks/object_detector/android_figurine/validation"  # @param {type:"string"}
test_data_path = ""  # @param {type:"string"}
data_format = "coco"  # @param ["coco", "pascal_voc"]

### 设置微调选项

您可以在不同的模型架构之间进行选择，以进一步定制您的训练：

- MobileNet-V2
- MobileNet-MultiHW-AVG

要设置模型架构和其他训练参数，请调整以下数值：

In [ ]:
model_architecture = "mobilenet_v2"  # @param ["mobilenet_v2", "mobilenet_multihw_avg"]

# The learning rate to use for gradient descent training.
learning_rate: float = 0.01  # @param {type:"number"}
# Batch size for training.
batch_size: int = 2  # @param {type:"number"}
# Number of training iterations over the dataset.
epochs: int = 10  # @param {type:"slider", min:0, max:100, step:1}
# If true, the base module is trained together with the classification layer on
# top.
do_fine_tuning: bool = False  # @param {type:"boolean"}
# A regularizer that applies a L1 regularization penalty.
l1_regularizer: float = 0.0  # @param {type:"number"}
# A regularizer that applies a L2 regularization penalty.
l2_regularizer: float = 0.0001  # @param {type:"number"}
# A boolean controlling whether the training dataset is augmented by randomly
# distorting input images, including random cropping, flipping, etc. See
# utils.image_preprocessing documentation for details.
do_data_augmentation: bool = True  # @param {type:"boolean"}
# Number of training samples used to calculate the decay steps
# and create the training optimizer.
decay_samples: int = 2560000  # @param {type:"number"}
# Number of warmup steps for a linear increasing warmup schedule on learning
# rate. Used to set up warmup schedule by model_util.WarmUp.
warmup_epochs: int = 2  # @param {type:"number"}
# The number of epochs for cosine decay learning rate.
cosine_decay_epochs: int = 5  # @param {type:"number"}
# The alpha value for cosine decay learning rate.
cosine_decay_alpha: float = 5  # @param {type:"number"}

### 运行微调
准备好您的训练数据集和微调选项后，您就可以开始微调过程。这个过程消耗资源，根据您的可用计算资源可能需要几分钟到几个小时。在使用GPU处理的Vertex AI上，下面的示例微调大约需要3到4分钟。

要开始微调过程，请使用以下代码：

In [ ]:
model_export_path = EXPORTED_MODEL_OUTPUT_DIRECTORY
evaluation_result_path = EVALUATION_RESULT_OUTPUT_DIRECTORY

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAINING_MACHINE_TYPE,
            "accelerator_type": TRAINING_ACCELERATOR_TYPE,
            "accelerator_count": TRAINING_ACCELERATOR_COUNT,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAINING_CONTAINER,
            "command": [],
            "args": [
                "--task_name=object_detector",
                "--training_data_path=%s" % training_data_path,
                "--validation_data_path=%s" % validation_data_path,
                "--test_data_path=%s" % test_data_path,
                "--data_format=%s" % data_format,
                "--model_export_path=%s" % model_export_path,
                "--evaluation_result_path=%s" % evaluation_result_path,
                "--model_architecture=%s" % model_architecture,
                "--hparams=%s"
                % json.dumps(
                    {
                        "learning_rate": learning_rate,
                        "batch_size": batch_size,
                        "epochs": epochs,
                        "do_fine_tuning": do_fine_tuning,
                        "l1_regularizer": l1_regularizer,
                        "l2_regularizer": l2_regularizer,
                        "do_data_augmentation": do_data_augmentation,
                        "decay_samples": decay_samples,
                        "warmup_epochs": warmup_epochs,
                        "cosine_decay_epochs": cosine_decay_epochs,
                        "cosine_decay_alpha": cosine_decay_alpha,
                    }
                ),
            ],
        },
    }
]

training_job = aiplatform.CustomJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

training_job.run()

## 评估并导出模型

### 评估性能

如果您已经指定了测试数据，您可以在测试数据集上进行评估，并打印出损失和coco指标。评估模型性能最重要的指标通常是“AP” coco指标，即平均精度。

In [ ]:
def get_evaluation_result(evaluation_result_path):
    try:
        with tensorflow.io.gfile.GFile(evaluation_result_path, "r") as input_file:
            evalutation_result = json.loads(input_file.read())
        return evalutation_result["loss"], evalutation_result["coco_metrics"]
    except:
        print("Evaluation result not found. Did you provide a test dataset?")
        return None


evaluation_result = get_evaluation_result(EVALUATION_RESULT_OUTPUT_FILE)

if evaluation_result is not None:
    print(f"Validation loss: {evaluation_result[0]}")
    print(f"Validation coco metrics: {evaluation_result[1]}")

### 导出模型
在对模型进行微调和评估之后，您可以将其保存为Tensorflow Lite模型，尝试在MediaPipe Studio中的[Object Detector](https://mediapipe-studio.webapps.google.com/demo/object_detector)演示中使用，或者根据[Object detection task guide](https://developers.google.com/mediapipe/solutions/vision/object_detector)将其集成到您的应用程序中。导出的模型还包括元数据和标签映射。

In [ ]:
import sys


def copy_model(model_source, model_dest):
    ! gsutil cp {model_source} {model_dest}

copy_model(EXPORTED_MODEL_OUTPUT_FILE, "object_detection_model.tflite")

if "google.colab" in sys.modules:
    from google.colab import files

    files.download("object_detection_model.tflite")

整理

In [ ]:
# Delete training data and jobs.
if training_job.list(filter=f'display_name="{TRAINING_JOB_DISPLAY_NAME}"'):
    training_job.delete()

!gsutil rm -r {STAGING_BUCKET}